In [443]:
import pandas as pd
import numpy as np
import os
import sys
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
pd.options.mode.chained_assignment = None 

In [444]:
col_dirs = os.listdir('Collisions/')
print(col_dirs)

list_of_df = []
for col_run in col_dirs:
    list_of_df.append(pd.read_csv('Collisions/'+col_run+'/'+col_run+'_final.csv'))
    
collisions_data = pd.concat(list_of_df)
collisions_data = collisions_data.reset_index(drop=True)

['320917', '322179', '324980', '322022', '324878', '321305', '324293', '324970', '321457', '321475']


In [445]:
collisions_data.head(5)

,run,chamber,wheel,sector,station,avg_cluster_size,occupancy,avg_bx_dist,avg_no_of_clusters,avg_multiplicity,lumisections,type,avg_efficiency,fid_eff_ch_level
0,320917,W-2_RB1in_S01,-2,1,1,1.49793,59592,0.008220,1.00361,1.50333,1925,Col,45.593319,78.731670
1,320917,W-2_RB1out_S01,-2,1,1,1.47164,53391,0.009950,1.00312,1.47624,1925,Col,51.693338,85.704300
2,320917,W-2_RB2in_S01,-2,1,2,1.56267,66112,0.010613,1.00334,1.56790,1925,Col,73.506504,89.979511
3,320917,W-2_RB2out_S01,-2,1,2,1.48085,58812,0.013395,1.00285,1.48508,1925,Col,78.429471,92.924316
4,320917,W-2_RB3+_S01,-2,1,3,1.49392,27509,0.008200,1.00120,1.49570,1925,Col,86.473099,91.954258


# Analyzing the records with zero fiducial efficiency

In [446]:
zero_fid_rows = collisions_data.loc[collisions_data['fid_eff_ch_level'] == 0]

# Calculating the percentage of the zero-efficiency valued records in the dataset
data_size = collisions_data.shape[0]
zero_data_size = zero_fid_rows.shape[0]
zero_percentage = round((zero_data_size/data_size)*100,2)
print("Dataset for all the chambers contains approximately "+str(zero_percentage)+"% of records with 0 fiducial efficiency\n\n")

# Records with zero-valued fid efficiency
print(zero_fid_rows)

Dataset for all the chambers contains approximately 8.19% of records with 0 fiducial efficiency


         run         chamber  wheel  sector  station  avg_cluster_size  \
7     320917    W-2_RB4-_S01     -2       1        4           0.00000   
12    320917    W-2_RB3+_S02     -2       2        3           0.00000   
21    320917    W-2_RB3-_S03     -2       3        3           0.00000   
33    320917    W-2_RB4-_S04     -2       4        4           0.00000   
41    320917    W-2_RB4-_S05     -2       5        4           0.00000   
...      ...             ...    ...     ...      ...               ...   
4756  321475   W+2_RB2in_S07      2       7        2           2.03058   
4757  321475  W+2_RB2out_S07      2       7        2           1.86599   
4768  321475    W+2_RB4+_S08      2       8        4           0.00000   
4790  321475    W+2_RB3-_S11      2      11        3           0.00000   
4797  321475    W+2_RB3-_S12      2      12        3           0.00000   

      occupan

### 1. Looking at incorrect zero efficiency valued records

In [447]:
# Collecting records with fiducial efficiency equal to 0 when some 
# other attributes contains non-zero values. Meaning there are
# some discrepancies in efficiency computation since the chamber
# contains metrics that would otherwise result in fiducial efficiency
# greater than 0.

incorrect_zero_efficiency = zero_fid_rows[(zero_fid_rows['avg_cluster_size'] != 0) | (zero_fid_rows['occupancy'] != 0) | (zero_fid_rows['avg_bx_dist'] != 0) | (zero_fid_rows['avg_no_of_clusters'] != 0) | (zero_fid_rows['avg_multiplicity'] != 0)]
incorrect_zero_efficiency.head(5)

,run,chamber,wheel,sector,station,avg_cluster_size,occupancy,avg_bx_dist,avg_no_of_clusters,avg_multiplicity,lumisections,type,avg_efficiency,fid_eff_ch_level
436,320917,W+2_RB2in_S07,2,7,2,1.63335,65522,0.011467,1.00323,1.63862,1925,Col,0.0,0.0
437,320917,W+2_RB2out_S07,2,7,2,1.52535,54368,0.012064,1.00228,1.52882,1925,Col,0.0,0.0
916,322179,W+2_RB2in_S07,2,7,2,2.02692,88465,0.009463,1.00361,2.03424,1788,Col,0.0,0.0
917,322179,W+2_RB2out_S07,2,7,2,1.87182,73817,0.008901,1.00254,1.87658,1788,Col,0.0,0.0
1396,324980,W+2_RB2in_S07,2,7,2,2.03772,97301,0.008984,1.00353,2.04491,2325,Col,0.0,0.0


In [448]:
# As can be seen, almost all of the errors that occurred are
# computed of the chamber in the exact same location. It can be
# concluded that computation of fiducial efficiency is incorrect 
# when computing efficiency for Wheel +2, Sector 7, Station 2 IN and OUT
# chambers. Thus, needs further investigation, as all of these chambers
# for all the runs (20 chambers in total) contains incorrect values.

# The values should be probably removed when applying machine learning 
# models as they are incorrect. However,
# all the other records containing 0 fid efficiency seems to be appropriate,
# as all the other metrics are also zero.
print(" Chambers of incorrectly computed fiducial efficiency:")
print(incorrect_zero_efficiency['chamber'].values)



# There is also this one chamber which seems to have incorrect
# data assigned to him, such as values of 41 for avg_cluster size,
# occupancy, avg_multiplicity and so on. Can say that it's an 
# noise in the data that has to be removed as well for further 
# modeling. 
print("\n\n An outlier chamber containing fiducial efficiency of 0:")
incorrect_zero_efficiency[incorrect_zero_efficiency['chamber']=='W-1_RB2out_S07']

 Chambers of incorrectly computed fiducial efficiency:
['W+2_RB2in_S07' 'W+2_RB2out_S07' 'W+2_RB2in_S07' 'W+2_RB2out_S07'
 'W+2_RB2in_S07' 'W+2_RB2out_S07' 'W+2_RB2in_S07' 'W+2_RB2out_S07'
 'W-1_RB2out_S07' 'W+2_RB2in_S07' 'W+2_RB2out_S07' 'W+2_RB2in_S07'
 'W+2_RB2out_S07' 'W+2_RB2in_S07' 'W+2_RB2out_S07' 'W+2_RB2in_S07'
 'W+2_RB2out_S07' 'W+2_RB2in_S07' 'W+2_RB2out_S07' 'W+2_RB2in_S07'
 'W+2_RB2out_S07']


 An outlier chamber containing fiducial efficiency of 0:


,run,chamber,wheel,sector,station,avg_cluster_size,occupancy,avg_bx_dist,avg_no_of_clusters,avg_multiplicity,lumisections,type,avg_efficiency,fid_eff_ch_level
2069,324878,W-1_RB2out_S07,-1,7,2,41.0,41,1.0,1.0,41.0,1800,Col,0.0,0.0


### 2. Looking at possibly correct zero efficiency valued records

In [449]:
# Collecting records with fiducial efficiency equal to 0 when all the 
# other attributes (avg_cluster_size, occupancy, etc.) also contains
# zero values. Meaning the fiducial efficiency of zero for such chambers
# should be correct.

correct_zero_efficiency = zero_fid_rows[(zero_fid_rows['avg_cluster_size'] == 0) & (zero_fid_rows['occupancy'] == 0) & (zero_fid_rows['avg_bx_dist'] == 0) & (zero_fid_rows['avg_no_of_clusters'] == 0) & (zero_fid_rows['avg_multiplicity'] == 0)]
correct_zero_efficiency.head(5)

,run,chamber,wheel,sector,station,avg_cluster_size,occupancy,avg_bx_dist,avg_no_of_clusters,avg_multiplicity,lumisections,type,avg_efficiency,fid_eff_ch_level
7,320917,W-2_RB4-_S01,-2,1,4,0.0,0,0.0,0.0,0.0,1925,Col,0.0,0.0
12,320917,W-2_RB3+_S02,-2,2,3,0.0,0,0.0,0.0,0.0,1925,Col,0.0,0.0
21,320917,W-2_RB3-_S03,-2,3,3,0.0,0,0.0,0.0,0.0,1925,Col,0.0,0.0
33,320917,W-2_RB4-_S04,-2,4,4,0.0,0,0.0,0.0,0.0,1925,Col,0.0,0.0
41,320917,W-2_RB4-_S05,-2,5,4,0.0,0,0.0,0.0,0.0,1925,Col,0.0,0.0


In [450]:
# It can be clearly seen that chambers in the outer layers
# (station 4 or 3) tend to have more zero valued fiducial
# efficiency rather than chambers in the first two Stations
# (Probably reasonable? as the detectors are further away
# and it's safe to assume they might be detecting less muons)
print("Stations with zero efficiency distribution:")
print(correct_zero_efficiency['station'].value_counts())

# There is no strong correlation between 0 fiducial efficiency
# and Sector numbers. However, Sectors 1 and 7 at the top
# seems reasonable based on their perpendicular position
# in the detector, receiving less cosmic muons?
# (however, what about sector 4 with most 0 efficiency records??)
print("\nSectors with zero efficiency distribution:")
print(correct_zero_efficiency['sector'].value_counts())


# No strong correlation between wheel and 0 fiducial efficiency
# whatsoever, except for the negative side of the barrel
# containing more 0-efficiency chambers.
print("\n Wheels with zero efficiency distribution:")
print(correct_zero_efficiency['wheel'].value_counts())

Stations with zero efficiency distribution:
4    252
3    103
2     17
Name: station, dtype: int64

Sectors with zero efficiency distribution:
4     93
1     60
7     55
8     40
5     32
2     30
12    30
11    20
3     12
Name: sector, dtype: int64

 Wheels with zero efficiency distribution:
-1    165
-2     85
 0     52
 1     40
 2     30
Name: wheel, dtype: int64


### 3. Conclusion on zero efficiency valued records

Assuming that the records analyzed in 1. are damaged and they cannot be used for further computations it is safe to believe that the chamber will have the fiducial efficiency of 0 if and only if all the other attributes of such chamber contain 0 values, as inspected above in 2.


Having this in mind, we might consider dropping all the records containing zero fiducial efficiency value as they won't provide us with any relevant information when calculating the efficiency of a chamber, since only the records containing all the attributes (avg_cluster_size, occupancy, avg_bx_dist, avg_no_of_clusters, avg_multiplicity) of value zero will have a final fiducial efficiency equal to 0. 

Moreover, the data suggests that some actual fiducial efficiency values start at 40 (see cell below), thus, it is possible to assume that the 'correct' records with fiducial efficiency of 0 might be also discrepancies of the detectors, since the other fiducial efficiency values are nowhere close to 0 (closest one to be over 40).

In [451]:
# Looking at the histograms in EDA_Collisions we can see that the dataset
# contains almost no records that have fiducial efficiency greater that 0 
# and less than 40. To make sure, we check the exsiting data:

eff_0_to_40 = collisions_data[(collisions_data['fid_eff_ch_level']>0)&(collisions_data['fid_eff_ch_level']<40)]
print(eff_0_to_40)
print("\n There are",eff_0_to_40.shape[0],"records containing fid efficiency between 0 and 40 in the dataset")

# As we can see, no fiducial efficiency values between 0 and 40 exists for this dataset.

Empty DataFrame
Columns: [run, chamber, wheel, sector, station, avg_cluster_size, occupancy, avg_bx_dist, avg_no_of_clusters, avg_multiplicity, lumisections, type, avg_efficiency, fid_eff_ch_level]
Index: []

 There are 0 records containing fid efficiency between 0 and 40 in the dataset


# -------- Problem formulation --------

As the problem itself suggests, predicting the efficiency of a chamber makes it a regression problem. As for regression model, it tends to be more difficult to minimize the objective function of such model. Therefore, it is decided to firstly try to solve a classification problem, splitting the dataset into N classes based on the fiducial efficiency into N equal sized buckets and trying out multiple classification models to see what output can be expected. 

Later on, the data will be used to train the regression models to be able to predict the numerical value for fiducial efficiency for a single chamber. 

Such model would help to quickly approximate the fiducial efficiency of a chamber after a Collision or a Cosmic run based on the initial metrics collected from the detectors (such as avg_cluster_size, occupancy, etc.). Also, the fiducial efficiency could be approximated for the chambers that were incorrectly assigned with the fiducial efficiency of 0 (such as the records analyzed in 1.) or if the 'correct' zero-valued records (analyzed in 2.) appears to be a discrepancy as well, such chambers could be assigned by a fiducial efficiency approximation of such model.

Modeling decisions will be based on the data analysis performed in **EDA_Collisions.ipynb** notebook.

# Prepare data for classification task (drop unnecessary columns, assign labels, standardize, shuffle, split into train/test/validate.)

Some attributes in the dataset are not useful for training the model. Such attributes would be:
* run (run number should not be an indicator for the efficiency computation)
* chamber (chamber name will be removed as all the chamber location data is stored within wheel, sector, station attributes)
* type (all the records used are Collisions type, therefore, the column has to be dropped)
* avg_efficiency (this is an efficiency retrieved from DQM system for the chambers, however, the project's aim is to compute a fiducial efficiency, thus, the column will have to be dropped)

The data will have to be split into N classes (buckets) based on their fiducial efficiency values to have an equal distribution for each class. 
For this case we'll try to use the existing data and split it into equal sized buckets based on the efficiency for current data. However, if there are some more substantial ranges of efficiency that could be split into, for example, weak efficiency, medium efficiency, and strong efficiency classes, we could augment data for each range to have an equal distribution. However, at the moment, it will be split into 3 classes by dividing existing data into equally distributed 3 buckets (see below, it gets us ranges (40.7, 91.2] for weak efficiency, (91.2, 97,2] for medium efficiency and (97.2, 100] for strong efficiency). The classes will be assigned with labes 0, 1 and 2 respectively.

As mentioned above, the zero valued records will have to be removed so that the model could use only the most correct data to be able to predict the efficiency as best as possible.

### Dropping unnecessary columns and filtering records with non zero efficiency

In [452]:
# Unnecessary columns are dropped

print('original shape:',collisions_data.shape)
shaped_data = collisions_data.copy()
shaped_data.drop(['run', 'chamber', 'type', 'avg_efficiency'], axis=1, inplace=True)

print('dropped shape:',shaped_data.shape)
shaped_data.head(5)

original shape: (4800, 14)
dropped shape: (4800, 10)


,wheel,sector,station,avg_cluster_size,occupancy,avg_bx_dist,avg_no_of_clusters,avg_multiplicity,lumisections,fid_eff_ch_level
0,-2,1,1,1.49793,59592,0.008220,1.00361,1.50333,1925,78.731670
1,-2,1,1,1.47164,53391,0.009950,1.00312,1.47624,1925,85.704300
2,-2,1,2,1.56267,66112,0.010613,1.00334,1.56790,1925,89.979511
3,-2,1,2,1.48085,58812,0.013395,1.00285,1.48508,1925,92.924316
4,-2,1,3,1.49392,27509,0.008200,1.00120,1.49570,1925,91.954258


In [453]:
# The data with unnecessary columns removed will now be filtered to
# keep only records that contain the fiducial efficiency greater than 0.

data_with_eff = shaped_data.loc[shaped_data['fid_eff_ch_level'] > 0]
print(data_with_eff.shape)


# Get the range of the bins to split the data into three classes based
# on their fiducial efficiency values.

bins = pd.qcut(data_with_eff['fid_eff_ch_level'],3)
print(bins.value_counts())
interval_values = bins.cat.categories.right
print('Class splitting interval values: ',interval_values[0], interval_values[1], interval_values[2])

(4407, 10)
(40.718, 91.22]    1469
(91.22, 97.227]    1469
(97.227, 100.0]    1469
Name: fid_eff_ch_level, dtype: int64
Class splitting interval values:  91.22 97.227 100.0


### Labels based on the efficiency value are added

In [454]:
# Creating a label column for the dataset, splitting into 3 equally distributed
# classes. label 0 - if efficiency is less than or equal to 91.22
# label 1 - if efficiency is greater than 91.22 and less than or equal to 97.22
# label 2 - if efficiency is greather than 97.22

def assign_label(fid_eff_val):
    if fid_eff_val <= interval_values[0]:
        return 0
    elif fid_eff_val <= interval_values[1]:
        return 1
    else:
        return 2
    
data_with_eff['label'] = data_with_eff['fid_eff_ch_level'].apply(lambda x: assign_label(x))

In [455]:
print(data_with_eff.shape)
print(data_with_eff['label'].value_counts())
data_with_eff.head(5)

(4407, 11)
0    1469
1    1469
2    1469
Name: label, dtype: int64


,wheel,sector,station,avg_cluster_size,occupancy,avg_bx_dist,avg_no_of_clusters,avg_multiplicity,lumisections,fid_eff_ch_level,label
0,-2,1,1,1.49793,59592,0.008220,1.00361,1.50333,1925,78.731670,0
1,-2,1,1,1.47164,53391,0.009950,1.00312,1.47624,1925,85.704300,0
2,-2,1,2,1.56267,66112,0.010613,1.00334,1.56790,1925,89.979511,0
3,-2,1,2,1.48085,58812,0.013395,1.00285,1.48508,1925,92.924316,1
4,-2,1,3,1.49392,27509,0.008200,1.00120,1.49570,1925,91.954258,1


### Shuffle and  split data into Train/Validate/Test datasets

In [456]:
# Firstly, the column with fid_eff_ch_level will be dropped 
# since we're firstly training a classifier to predict one of the
# three classes that were assgined as labels 0, 1 and 2.
data_with_eff.drop(['fid_eff_ch_level'], axis=1, inplace=True)

# Also, shuffle the rows in the dataset
data_with_eff = shuffle(data_with_eff)

data_with_eff.head(5)

,wheel,sector,station,avg_cluster_size,occupancy,avg_bx_dist,avg_no_of_clusters,avg_multiplicity,lumisections,label
3693,1,6,2,1.91299,102389,0.004372,1.00403,1.92071,2204,1
3665,1,3,1,1.52063,84509,0.032137,1.00425,1.52709,2204,0
3342,2,10,3,1.81240,37812,0.004170,1.00072,1.81370,2346,2
2722,1,5,1,1.70664,141549,0.010080,1.00626,1.71733,2649,1
1859,2,5,1,2.32370,94196,0.081407,1.00312,2.33095,1813,2


In [457]:
# Separate the data into attributes dataFrame and
# labels dataFrame. X_data and Y_data
x_data = data_with_eff.iloc[:, 0:9]
y_data = data_with_eff.iloc[:,-1:]

# Split the data into Train/Validate/Test datasets.
# Train dataset will be used to train models, the data
# the models will be learning from. Validation set to be used
# for tuning the models, for finding the best hyperparameters. 
# Whereas Test dataset will be used for final evaluation once the
# models are optimized the most.

train_ratio = 0.7
validation_ratio = 0.15
test_ratio = 0.15

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=1-train_ratio, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=1)

print("Dataset is split into Train/Validate/Test subsets of size:")
print("train - %.2f%%" % ((x_train.shape[0]/(x_train.shape[0]+x_val.shape[0]+x_test.shape[0]))*100))
print("validate - %.2f%%" % ((x_val.shape[0]/(x_train.shape[0]+x_val.shape[0]+x_test.shape[0]))*100))
print("test - %.2f%%" % ((x_test.shape[0]/(x_train.shape[0]+x_val.shape[0]+x_test.shape[0]))*100))


Dataset is split into Train/Validate/Test subsets of size:
train - 69.98%
validate - 15.00%
test - 15.02%


### Standardize data

In [458]:
# The data will be standardized so that the different
# attributes in the dataset will be within the same scale.
# So that one attribute would not contain more weight 
# than other attribute.

# Standardization transforms data such that its distribution
# whill have mean value of 0 and a standard deviation of 1. 
# It is performed feature/column wise. 
# x = (x-μ)/σ

# validation and test data will have to be standardized before testing
# by using mean and standard deviation of Train dataset.

#define scaler
scaler = preprocessing.StandardScaler().fit(x_train)
print(scaler.mean_)

# transform train data
x_train = scaler.transform(x_train)


# In practice, the scaler is stored in the preprocessing pipeline
# and new data arriving to the model will be scaled before
# sent into model black box to get prediction. However, right now,
# the validation data will be scaled at the same time.

x_val = scaler.transform(x_val)

[5.93385214e-02 6.44066148e+00 2.42542153e+00 1.80563526e+00
 7.36054063e+04 1.47939634e-02 1.00269067e+00 1.81067119e+00
 2.07849676e+03]


# Models

In [459]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [460]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(x_train,y_train)

knn_pred = model.predict(x_val)
knn_accuracy = accuracy_score(y_val, knn_pred)

print("KNN Accuracy: ", knn_accuracy)

KNN Accuracy:  0.7276853252647504
